In [12]:
import geopandas as gpd
import richdem as rd
import rasterio
import numpy as np
import glob
import pandas as pd
import os
from pathlib import Path
import math
import geopandas as gpd
from shapely.geometry import Point
import multiprocessing as mp
import time
from datetime import datetime
import math
from numba import jit

In [13]:
def plot(data,color = 'jet'):
    rd.rdShow(data,axes=False, cmap=color, figsize=(9, 6))

In [14]:
def write_to_csv(file_name,data):
    dir_ = str(file_name)+'.csv'
    if os.path.isfile(dir_):
        with open(dir_, 'a') as f:
            data.to_csv(f, index=False, header=False)
    else:
        with open(dir_, 'a') as f:
            data.to_csv(f, index=False, header=True)

In [15]:
@jit(nopython=True)
def check_not_null(m):
    l = []
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if m[i,j]!=0:
                l.append([i,j])
    return l

In [24]:
# @jit
def distance(i,j):
    return (i[0]-j[0])**2+(i[1]-j[1])**2

In [25]:
@jit
def haversine(loc1, loc2):

    # convert decimal degrees to radians 
    lon1, lat1 = map(math.radians, loc1)
    lon2, lat2 = map(math.radians, loc2)
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    m = 6367000 * c
    return m

print(haversine( (-70.92643518584309, 42.167083332676334), (-70.917175926573876 ,42.157916665999807)))

1272.533652906026


In [26]:
@jit(nopython=True)
def min_val_pos(v,l):
    val = 999999
    pos = None
    for i in l:
        j = distance(v,i)
        if j<val:
            val = j
            pos = i
    return pos

In [27]:
def dataToCSV(data,df):

    gp_df = gpd.GeoDataFrame(data, crs="EPSG:4326",geometry='geometry')
    df = gpd.GeoDataFrame(df, crs="EPSG:4326",geometry='geometry')
    
    gp_df['centroid'] = gp_df['geometry']
    
    js = gpd.sjoin(df[['FIPS','geometry']], gp_df, how="left", op='contains')
    
    js.drop('geometry',axis=1,inplace=True)
    js.drop('index_right',axis=1,inplace=True)
    js.rename(columns = {'centroid':'geometry'},inplace=True)
    js.dropna(inplace=True)
    
    for i in js.groupby('FIPS'):
        write_to_csv(i[0],i[1])

In [28]:
from concurrent.futures import ProcessPoolExecutor

def multiprocessing(func, args, workers):
    with ProcessPoolExecutor(max_workers=workers) as executor:
        res = executor.map(func, args)
    return list(res)

In [29]:
# @jit(parallel = True)
def vdf_hfd(path):

    beau = rasterio.open(path)
    madeenah_rio_band1 = beau.read(1).astype('float64')
    madeenah_richdem = rd.rdarray(madeenah_rio_band1, no_data=-9999)
    

    
    rd.FillDepressions(madeenah_richdem, epsilon=True, in_place=True)
    accum_d8 = rd.FlowAccumulation(madeenah_richdem, method='Quinn')

    accum_d8[accum_d8<150]=0
    accum_d8[accum_d8!=0]=1
    m = accum_d8*madeenah_richdem

  
    
    l = check_not_null(m)
    dim = m.shape

    hfd = np.zeros(shape=(dim[0],dim[1]))
    vfd = np.zeros(shape=(dim[0],dim[1]))
    
    madeenah_rich_slope = rd.TerrainAttribute(madeenah_richdem, attrib='slope_degrees').round(2)
    madeenah_rich_aspect = rd.TerrainAttribute(madeenah_richdem, attrib='aspect').round(2)
    
    accum_8 = rd.FlowAccumulation(madeenah_richdem, method='D8')
    
    try:

        for i in range(dim[0]):
            for j in range(dim[1]):
                if m[i,j]==0:
                    d = min_val_pos((i,j),l)
                    vfd[i,j] = m[d[0],d[1]]
                    hfd[i,j] = haversine(beau.xy(i,j),beau.xy(d[0],d[1]))
                else:
                    vfd[i,j] = m[i,j]


        vfd =  madeenah_richdem-vfd
        vfd[vfd<0]=0

        hfd = rd.rdarray(hfd, no_data=-9999)
        vfd = rd.rdarray(vfd, no_data=-9999)


        pnt_data = {'elevation':[],'vfd':[],'slope':[],'aspect':[],'upslopeArea':[],'twi':[],'hfd':[],'geometry':[]}
        for i in range(0,m.shape[0]):
            for j in range(0,m.shape[1]):
                pnt_data['elevation'].append(madeenah_richdem[i,j].round(2))
                pnt_data['geometry'].append(Point(beau.xy(i,j)))
                pnt_data['vfd'].append(vfd[i,j].round(2))
                pnt_data['hfd'].append(hfd[i,j].round(2))
                pnt_data['slope'].append(madeenah_rich_slope[i,j].round(2))
                pnt_data['aspect'].append(madeenah_rich_aspect[i,j].round(2))
                
                pnt_data['upslopeArea'].append(accum_8[i,j].round(2))
                twi = math.log((accum_8[i,j]+10)/(math.tan(madeenah_rich_slope[i,j]*0.01746)+0.01))
                pnt_data['twi'].append(round(twi,2))
        


        dataToCSV(pnt_data,censusRaster)
#         os.remove(path)
        
    except:
        print('ERROR : {}'.format(path))
#         os.rename(path,'/home/cyberboxer_v10/rr/distanceCalculation/rasterProcessed/'+path.split('/')[-1])

In [ ]:
%%time
import time

if __name__=="__main__":
        
    global censusRaster
    
    censusRaster = gpd.read_file('ny_census-shape_file.geojson')
    censusRaster = censusRaster[['FIPS','geometry']]
        
    for file in glob.glob('result/*.tif'):
        s = time.time()
        vdf_hfd(file)
        print(time.time()-s)
